In [ ]:
# -*- coding: utf-8 -*-

"""
Project: RRI Model Parameter Calibration using SCE-UA
Description:
    This script implements the Shuffled Complex Evolution (SCE-UA) global optimization 
    algorithm to automatically calibrate sensitive parameters for the Rainfall-Runoff-Inundation (RRI) model.

Overall Workflow:
    1. Configuration: Define file paths, observation coordinates, rainfall events, parameter boundaries, and fixed parameters.
    2. Initialization: Instantiate the SCE-UA optimizer with hyperparameters (population size, number of complexes, etc.).
    3. Optimization Loop:
       a. Sampling: Generate an initial population or evolve offspring within the parameter space.
       b. Model Execution (run_model_and_evaluate):
          - Update 'RRI_Input.txt' with the current parameter set.
          - Execute the external RRI model (.exe).
          - Read the generated raster output files.
          - Extract time-series data at specific observation coordinates.
          - Compare simulated results with observed data (Excel).
          - Calculate the Nash-Sutcliffe Efficiency (NSE).
          - Return -NSE (as the optimizer minimizes the objective function).
       c. Evolution: Sort population, partition into complexes, evolve using CCE (Simplex method), and shuffle.
    4. Output: Display and save the optimal parameter set and the best NSE score.

Notes:
    - Requires the GDAL library for raster data processing.
    - Ensure the RRI executable and input file formats are correct before running.
"""

import os
import re
import subprocess
import numpy as np
import pandas as pd
from osgeo import gdal

# ==========================================
# 1. Global Configuration & Parameters
# ==========================================

# 1.1 File Paths and Directory Settings
PATHS = {
    'dem': "topo/dem_250.tif",       # Path to DEM file, used for raster reference info
    'out_folder': "out",             # Folder for raw RRI model outputs
    'result_folder': "results",      # Folder to save processed results
    'out_prefix': "qr",              # Prefix of RRI output files (e.g., qr_1.out)
    'rri_exe': "0_rri_1_4_2_6.exe",  # RRI model executable filename
    'rri_input': "RRI_Input.txt",    # Path to the main RRI configuration file
    'obs_data': "./results/obsQ.xlsx", # Path to observed discharge data (Excel)
    'rain_folder': "./rain"          # Folder containing rainfall data files
}

# 1.2 Observation Coordinates (Longitude, Latitude)
# The model extracts simulation results at these locations for comparison
LOCATIONS = [
    # (120.5, 30.2),       # Example Location 1
    (136.16375, 36.0081)   # Example Location 2
]

# 1.3 Rainfall Events List (Filename, Simulation Duration in Hours)
# Used for multi-event calibration. Format: (Rainfall Filename, Duration Hours)
RAIN_EVENTS = [
    ('rain_merged_precipitation_20040717_to_20040723.dat', 168),
    ('rain_merged_precipitation_20041019_to_20041022.dat', 81),
    ('rain_merged_precipitation_20050630_to_20050709.dat', 240),
    ('rain_merged_precipitation_20110707_to_20110711.dat', 120),
    ('rain_merged_precipitation_20110919_to_20110926.dat', 192),
    ('rain_merged_precipitation_20171021_to_20171028.dat', 192),
    ('rain_merged_precipitation_20180704_to_20180711.dat', 192)
]

# 1.4 Parameters to Optimize and Boundaries (Name: [Min, Max])
# The SCE-UA algorithm will search for optimal values within these ranges.
# Definitions align with the specific RRI model calibration framework.
PARAMS_BOUNDS = {
    # River Channel Parameter
    'ns_river':    [0.01, 0.1],       # River channel Manning’s roughness coefficient

    # Land Use Type 1 (e.g., Forest)
    'ns_slope_1':  [0.03, 1.0],       # Ground surface Manning’s roughness coefficient (Type 1)
    'soildepth_1': [0.05, 3.0],       # Soil depth (Type 1)
    'gammaa_1':    [0.3, 0.5],        # Soil effective porosity (Type 1)
    'gammam_1':    [0.05, 0.49],      # Soil capillary porosity (Type 1)
    'ka_1':        [0.01, 0.5],       # Lateral saturated hydraulic conductivity (Type 1)
    'beta_1':      [2.0, 30.0],       # Parameter for calculating the lateral hydraulic conductivity of the unsaturated zone (Type 1)
    

    # Land Use Type 2 (e.g., Non-Forest)
    'ns_slope_2':  [0.03, 1.0],       # Ground surface Manning’s roughness coefficient (Type 2)
    'soildepth_2': [0.05, 3.0],       # Soil depth (Type 2)
    'gammaa_2':    [0.3, 0.5],        # Soil effective porosity (Type 2)
    'gammam_2':    [0.05, 0.49],      # Soil capillary porosity (Type 2)
    'ka_2':        [0.01, 0.5],       # Lateral saturated hydraulic conductivity (Type 2)
    'beta_2':      [2.0, 30.0],       # Parameter for calculating the lateral hydraulic conductivity of the unsaturated zone (Type 2)
}

# 1.5 Fixed Parameters (Not involved in optimization)
# These values remain constant during the simulation runs
FIXED_PARAMS = {
    'ksv_1':       0.0000,            # Lateral saturated hydraulic conductivity (Type 1)
    'ksv_2':       0.0000,            # Lateral saturated hydraulic conductivity (Type 2)
    'faif_1':      0.0000,            # Wetting front suction head (Green-Ampt parameter) (Type 1)
    'faif_2':      0.0000             # Wetting front suction head (Green-Ampt parameter) (Type 2)
}

# ==========================================
# 2. SCE-UA Optimization Algorithm Class
# ==========================================

class SCEUA:
    """
    Implementation of the SCE-UA (Shuffled Complex Evolution - University of Arizona) algorithm.
    It is a heuristic global optimization method widely used in hydrological model calibration.
    """
    def __init__(self, objective_function, bounds, max_evaluations=10000, num_complexes=5,
                 points_per_complex=21, num_points=105, num_evolution_steps=21, 
                 num_offspring=1, alpha=0.8, beta=0.45):
        """
        Initialize SCE-UA algorithm parameters.
        
        Args:
            objective_function (callable): The function to minimize. Takes parameters, returns scalar.
            bounds (list/array): Parameter boundaries [[min, max], ...].
            max_evaluations (int): Maximum number of function evaluations allowed.
            num_complexes (int): Number of complexes (p).
            points_per_complex (int): Number of points in each complex (m).
            num_points (int): Total population size (s = p * m).
            num_evolution_steps (int): Number of evolution steps within each complex.
            num_offspring (int): Number of offspring generated per step.
            alpha (float): Reflection coefficient for the simplex method.
            beta (float): Contraction coefficient for the simplex method.
        """
        self.objective_function = objective_function
        self.bounds = np.array(bounds)
        self.dim = len(bounds)
        self.max_evaluations = max_evaluations
        
        # SCE-UA Parameters
        self.num_complexes = num_complexes
        self.points_per_complex = points_per_complex
        self.num_points = num_points
        self.num_evolution_steps = num_evolution_steps
        self.num_offspring = num_offspring
        self.num_parents = min(2 * self.dim + 1, self.points_per_complex)
        self.alpha = alpha
        self.beta = beta
        
        # State variables
        self.evaluations = 0
        self.best_solution = None
        self.best_objective = float('inf')
        self.evolution_history = []
    
    def generate_initial_population(self):
        """Generate the initial population using random uniform sampling within bounds."""
        population = np.zeros((self.num_points, self.dim))
        objectives = np.zeros(self.num_points)
        
        for i in range(self.dim):
            population[:, i] = np.random.uniform(self.bounds[i, 0], self.bounds[i, 1], self.num_points)
        
        for i in range(self.num_points):
            objectives[i] = self.objective_function(population[i])
            self.evaluations += 1
            
            if objectives[i] < self.best_objective:
                self.best_objective = objectives[i]
                self.best_solution = population[i].copy()
            self.evolution_history.append(self.best_objective)
            
            if self.evaluations >= self.max_evaluations:
                break
        
        # Sort population by objective function value (ascending)
        sort_indices = np.argsort(objectives)
        return population[sort_indices], objectives[sort_indices]
    
    def partition_into_complexes(self, population, objectives):
        """
        Partition the sorted population into multiple complexes (Shuffling).
        Ensures each complex contains a mix of good and bad points.
        """
        complexes = []
        complex_objectives = []
        for i in range(self.num_complexes):
            idx = np.arange(i, self.num_points, self.num_complexes)
            complexes.append(population[idx].copy())
            complex_objectives.append(objectives[idx].copy())
        return complexes, complex_objectives
    
    def evolve_complex(self, complex_points, complex_objectives):
        """
        Execute the Competitive Complex Evolution (CCE) process.
        Uses the Simplex method (Nelder-Mead variant) to evolve points within a complex.
        Includes Reflection, Contraction, and Mutation steps.
        """
        n = len(complex_points)
        
        for _ in range(self.num_evolution_steps):
            if self.evaluations >= self.max_evaluations:
                break
            
            # Select parents based on triangular probability distribution (better points have higher probability)
            num_parents_to_select = min(self.num_parents, n)
            weights = np.array([2 * (n + 1 - j) / (n * (n + 1)) for j in range(1, n + 1)])
            weights = weights / np.sum(weights)
            
            parent_indices = np.random.choice(range(n), size=num_parents_to_select, replace=False, p=weights)
            parent_indices = np.sort(parent_indices)
            
            parents = complex_points[parent_indices]
            parent_objectives = complex_objectives[parent_indices]
            
            # Calculate centroid (excluding the worst point)
            centroid = np.mean(parents[:-1], axis=0)
            worst_point = parents[-1]
            
            # 1. Reflection Step
            reflected_point = centroid + self.alpha * (centroid - worst_point)
            # Enforce boundary constraints
            for i in range(self.dim):
                reflected_point[i] = max(self.bounds[i, 0], min(reflected_point[i], self.bounds[i, 1]))
            
            reflected_objective = self.objective_function(reflected_point)
            self.evaluations += 1
            self._update_best(reflected_point, reflected_objective)
            
            if reflected_objective < parent_objectives[-1]:
                parents[-1] = reflected_point
                parent_objectives[-1] = reflected_objective
            else:
                # 2. Contraction Step
                contracted_point = centroid + self.beta * (worst_point - centroid)
                contracted_objective = self.objective_function(contracted_point)
                self.evaluations += 1
                self._update_best(contracted_point, contracted_objective)
                
                if contracted_objective < parent_objectives[-1]:
                    parents[-1] = contracted_point
                    parent_objectives[-1] = contracted_objective
                else:
                    # 3. Mutation Step
                    random_point = np.zeros(self.dim)
                    for i in range(self.dim):
                        random_point[i] = np.random.uniform(self.bounds[i, 0], self.bounds[i, 1])
                    
                    random_objective = self.objective_function(random_point)
                    self.evaluations += 1
                    self._update_best(random_point, random_objective)
                    
                    parents[-1] = random_point
                    parent_objectives[-1] = random_objective
            
            # Update points in the complex
            complex_points[parent_indices] = parents
            complex_objectives[parent_indices] = parent_objectives
            
            # Re-sort the complex
            sort_indices = np.argsort(complex_objectives)
            complex_points = complex_points[sort_indices]
            complex_objectives = complex_objectives[sort_indices]
        
        return complex_points, complex_objectives

    def _update_best(self, point, obj):
        """Helper to update the global best solution."""
        if obj < self.best_objective:
            self.best_objective = obj
            self.best_solution = point.copy()
        self.evolution_history.append(self.best_objective)

    def optimize(self):
        """
        Execute the main optimization loop.
        
        Returns:
            best_solution (array): The optimal parameter set.
            best_objective (float): The best objective function value (Negative NSE).
            evolution_history (list): History of the best objective value over evaluations.
        """
        population, objectives = self.generate_initial_population()
        
        while self.evaluations < self.max_evaluations:
            # 1. Partition into complexes
            complexes, complex_objectives = self.partition_into_complexes(population, objectives)
            
            evolved_complexes = []
            evolved_objectives = []
            
            # 2. Evolve each complex independently
            for i in range(self.num_complexes):
                c_pts, c_objs = self.evolve_complex(complexes[i], complex_objectives[i])
                evolved_complexes.append(c_pts)
                evolved_objectives.append(c_objs)
            
            # 3. Recombine (Un-shuffle) the population
            population = np.zeros((self.num_points, self.dim))
            objectives = np.zeros(self.num_points)
            
            idx = 0
            for i in range(self.num_complexes):
                for j in range(len(evolved_complexes[i])):
                    if idx < self.num_points:
                        population[idx] = evolved_complexes[i][j]
                        objectives[idx] = evolved_objectives[i][j]
                        idx += 1
            
            # 4. Sort the new population
            sort_indices = np.argsort(objectives)
            population = population[sort_indices]
            objectives = objectives[sort_indices]
            
            print(f"Evaluation {self.evaluations}: Best NSE = {-self.best_objective:.6f}")
            
            # Convergence check (stop if standard deviation is very small)
            if np.std(objectives) < 1e-6:
                print("Algorithm converged!")
                break
        
        return self.best_solution, self.best_objective, self.evolution_history

# ==========================================
# 3. Helper Functions (File I/O & Utils)
# ==========================================

def format_fortran_line(par_name, params_dict, num_landuse):
    """
    Format a parameter line string for Fortran input (using 'd0' for double precision).
    
    Args:
        par_name: Parameter name tag (e.g., '# ns_slope').
        params_dict: Dictionary containing current parameter values.
        num_landuse: Number of land use types (determines values per line).
    """
    values = []
    base_name = par_name[2:].strip() # Remove '# '
    for j in range(num_landuse):
        param_key = f"{base_name}_{j+1}"
        if param_key in params_dict:
            values.append(f"{params_dict[param_key]:.6f}d0")
        else:
            print(f"Error: Parameter {param_key} not found in dictionary.")
            exit(1)
    return '\t'.join(values) + f"      # {base_name}\n"

def modify_rri_input_file(params_dict, rain_file_path, duration_hours):
    """
    Modify the 'RRI_Input.txt' file.
    Updates parameter values, rainfall file path, and simulation duration for the Fortran model.
    """
    try:
        with open(PATHS['rri_input'], 'r') as f:
            lines = f.readlines()
    except FileNotFoundError:
        print(f"Error: {PATHS['rri_input']} not found.")
        exit(1)
    
    # Construct full rainfall path
    full_rain_path = os.path.join(PATHS['rain_folder'], rain_file_path)
    
    # Update rain path and time settings (Relies on specific line numbers)
    lines[2] = f"{full_rain_path}\n"
    lines[9] = f"{duration_hours}    # lasth(hour)\n"
    lines[12] = f"{duration_hours}    # outnum [-]\n"

    # Retrieve number of land use types
    num_landuse = None
    for i, line in enumerate(lines):
        if "# num_of_landuse" in line:
            num_landuse = int(lines[i].split('#')[0].strip())
            break
    
    if num_landuse is None:
        raise ValueError("Could not find 'num_of_landuse' in RRI_Input.txt")
    
    # Update lines based on parameter dictionary
    target_params = ["# ns_slope", "# soildepth", "# gammaa", "# ksv", 
                     "# faif", "# ka", "# gammam", "# beta"]
    
    for i, line in enumerate(lines):
        if "# ns_river" in line and 'ns_river' in params_dict:
            lines[i] = f"{params_dict['ns_river']:.6f}d0     # ns_river\n"
        else:
            for tag in target_params:
                if tag in line:
                    lines[i] = format_fortran_line(tag, params_dict, num_landuse)
                    break
    
    with open(PATHS['rri_input'], 'w') as f:
        f.writelines(lines)

def get_raster_info(tif_path):
    """Read GeoTIFF metadata (columns, rows, and geotransform parameters)."""
    ds = gdal.Open(tif_path)
    if ds is None:
        raise Exception(f"Cannot open raster file: {tif_path}")
    return {
        'cols': ds.RasterXSize,
        'rows': ds.RasterYSize,
        'geotransform': ds.GetGeoTransform(),
        'dataset': ds
    }

def lonlat_to_xy(lon, lat, geotransform):
    """
    Convert Longitude/Latitude coordinates to raster pixel coordinates (Column/x, Row/y).
    """
    x_origin = geotransform[0]
    y_origin = geotransform[3]
    pixel_width = geotransform[1]
    pixel_height = geotransform[5]
    
    x = int((lon - x_origin) / pixel_width)
    y = int((lat - y_origin) / pixel_height)
    return x, y

def read_out_file(out_file):
    """
    Read the output matrix from a single RRI .out file.
    Typically ASCII Grid format without headers.
    """
    try:
        return np.loadtxt(out_file)
    except Exception:
        # Fallback manual parsing if loadtxt fails
        with open(out_file, 'r') as f:
            lines = [l.strip() for l in f if l.strip() and not l.strip().startswith('#')]
        return np.array([[float(v) for v in l.split()] for l in lines])

def extract_time_series(lon, lat, tif_path, out_folder, out_prefix):
    """
    Extract time-series data for a specific coordinate from a sequence of output files.
    
    Args:
        lon, lat: Target coordinates.
        tif_path: Reference DEM path for coordinate conversion.
        out_folder: Directory containing .out files.
        out_prefix: Prefix of output files.
    Returns:
        Dictionary containing time steps and values, or None if out of bounds.
    """
    try:
        raster_info = get_raster_info(tif_path)
    except Exception as e:
        print(e)
        return None

    x, y = lonlat_to_xy(lon, lat, raster_info['geotransform'])
    
    if not (0 <= x < raster_info['cols'] and 0 <= y < raster_info['rows']):
        print(f"Coordinates ({lon}, {lat}) represent out of DEM bounds.")
        return None
    
    # Find and sort output files
    all_files = os.listdir(out_folder)
    # Regex match for qr_1.out, qr_2.out, etc.
    pattern = re.compile(rf'{out_prefix}_(\d+)\.out')
    out_files = sorted(
        [os.path.join(out_folder, f) for f in all_files if pattern.match(f)],
        key=lambda x: int(pattern.search(os.path.basename(x)).group(1))
    )
    
    if not out_files:
        return None
    
    values = []
    steps = []
    
    for fpath in out_files:
        step = int(pattern.search(os.path.basename(fpath)).group(1))
        steps.append(step)
        data = read_out_file(fpath)
        
        # Note: numpy uses [row, col], i.e., [y, x]
        if y < data.shape[0] and x < data.shape[1]:
            values.append(data[y, x])
        else:
            values.append(np.nan)
            
    return {'time_steps': steps, 'values': values, 'lon': lon, 'lat': lat, 'x': x, 'y': y}

def batch_process_locations(locations, output_folder):
    """
    Batch process all observation locations, save results to CSV, and clean up temporary .out files.
    """
    os.makedirs(output_folder, exist_ok=True)
    
    results = []
    for lon, lat in locations:
        ts = extract_time_series(lon, lat, PATHS['dem'], PATHS['out_folder'], PATHS['out_prefix'])
        if ts:
            results.append(ts)
    
    # Save combined results
    dfs = []
    for res in results:
        dfs.append(pd.DataFrame({
            'time_step': res['time_steps'],
            'value': res['values'],
            'lon': res['lon'],
            'lat': res['lat']
        }))
    
    if dfs:
        pd.concat(dfs).to_csv(os.path.join(output_folder, 'time_series_data.csv'), index=False)
    
    # Cleanup temporary output files in the 'out' folder
    for f in os.listdir(PATHS['out_folder']):
        try:
            os.remove(os.path.join(PATHS['out_folder'], f))
        except OSError:
            pass
            
    return results

def get_simulated_values():
    """Execute data extraction logic and return simulated values as a 1D array."""
    batch_process_locations(LOCATIONS, PATHS['result_folder'])
    try:
        df = pd.read_csv(os.path.join(PATHS['result_folder'], "time_series_data.csv"))
        return df['value'].values
    except Exception:
        return np.array([])

# ==========================================
# 4. Objective Function & Model Execution
# ==========================================

def run_model_and_evaluate(params, param_names, fixed_params=None):
    """
    Core Objective Function: Runs the RRI model and calculates average NSE.
    
    Args:
        params (list): Current list of parameter values from SCE-UA.
        param_names (list): Corresponding parameter names.
        fixed_params (dict): Dictionary of fixed parameters.
        
    Returns:
        float: Negative Average NSE (-AvgNSE).
        Note: SCE-UA minimizes the function, but higher NSE is better, so we return negative NSE.
        Returns infinity if the run fails or data is missing.
    """
    # Merge optimized and fixed parameters
    params_dict = dict(zip(param_names, params))
    if fixed_params:
        params_dict.update(fixed_params)
    
    nses_sum = 0.0
    
    # Iterate through all rainfall events
    for index, (rain_file, duration) in enumerate(RAIN_EVENTS):
        
        # 1. Modify input file configuration
        modify_rri_input_file(params_dict, rain_file, duration)
        
        # 2. Run external executable
        try:
            # Ensure correct working directory for DLLs/Configs
            subprocess.run([PATHS['rri_exe']], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        except subprocess.CalledProcessError as e:
            print(f"Run Error in event {index}: {e}")
            return float('inf') # Return infinity indicating invalid parameters
        
        # 3. Retrieve simulated results
        sim_data = get_simulated_values()
        
        # 4. Get observation data (from Excel)
        try:
            obs_df = pd.read_excel(PATHS['obs_data'])
            # Assuming Excel columns are 1, 2, 3... corresponding to event index + 1
            obs_values = obs_df[index + 1].values
            obs_values = obs_values[~np.isnan(obs_values)] # Remove NaNs
        except Exception as e:
            print(f"Obs Data Error: {e}")
            return float('inf')
        
        # 5. Calculate NSE for the single event
        if len(sim_data) != len(obs_values) or len(obs_values) == 0:
            print(f"Data length mismatch: Sim {len(sim_data)} vs Obs {len(obs_values)}")
            return float('inf')
            
        mean_obs = np.mean(obs_values)
        denominator = np.sum((obs_values - mean_obs) ** 2)
        
        if denominator == 0:
            nse = -1e9 # Avoid division by zero, assign a very low value
        else:
            nse = 1 - np.sum((obs_values - sim_data) ** 2) / denominator
            
        print(f"  Event {index+1} NSE: {nse:.4f}")
        nses_sum += nse

    avg_nse = nses_sum / len(RAIN_EVENTS)
    print(f"Avg NSE: {avg_nse:.4f}")
    
    # Return negative value for minimization
    return -avg_nse

# ==========================================
# 5. Main Entry Point
# ==========================================

if __name__ == "__main__":
    # Prepare list of parameter names and boundaries
    param_names = list(PARAMS_BOUNDS.keys())
    # Remove fixed parameters from bounds just in case
    for p in FIXED_PARAMS:
        if p in PARAMS_BOUNDS:
            del PARAMS_BOUNDS[p]
            
    bounds = [PARAMS_BOUNDS[name] for name in param_names]

    # Define objective function wrapper (lambda), fixing inputs other than 'p'
    objective_func = lambda p: run_model_and_evaluate(p, param_names, FIXED_PARAMS)

    # Initialize SCE-UA Optimizer
    optimizer = SCEUA(
        objective_function=objective_func,
        bounds=bounds,
        max_evaluations=10000,     # Max number of evaluations
        num_complexes=5,          # Number of complexes
        points_per_complex=21,    # Points per complex
        num_points=105,           # Total points (usually complexes * points_per_complex)
        num_evolution_steps=21    # Internal evolution steps
    )

    print("Starting SCE-UA Optimization...")
    print(f"Optimizing {len(param_names)} parameters.")
    
    # Start optimization process
    best_params, best_obj, history = optimizer.optimize()

    # Output results
    print("\nOptimization Finished!")
    # Note: best_obj is negative NSE, negate it to show the real NSE
    print(f"Best Avg NSE: {-best_obj:.6f}")
    print("Best Parameters:")
    
    results_dict = FIXED_PARAMS.copy()
    for i, name in enumerate(param_names):
        print(f"{name}: {best_params[i]:.6f}")
        results_dict[name] = best_params[i]

    # Save optimal parameters to file
    with open('optimal_parameters.txt', 'w') as f:
        f.write("Optimal Parameters:\n")
        for name, value in results_dict.items():
            f.write(f"{name}: {value}\n")
        f.write(f"\nBest NSE: {-best_obj}\n")